In [19]:
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision.models import resnet18


In [20]:
# Preprocessing: normalization and data augmentation
transform_train = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomCrop(32, padding=4),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
])

# doesn't it crop and flip the test set?
transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
])

# Load the CIFAR-10 dataset
trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform_train)
testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform_test)

# Split the dataset into labeled and unlabeled data
labeled_indices = list(range(0, 25000))
unlabeled_indices = list(range(25000, 50000))

labeled_set = torch.utils.data.Subset(trainset, labeled_indices)
unlabeled_set = torch.utils.data.Subset(trainset, unlabeled_indices)

# Create data loaders
batch_size = 100
labeled_loader = torch.utils.data.DataLoader(labeled_set, batch_size=batch_size, shuffle=True, num_workers=2)
unlabeled_loader = torch.utils.data.DataLoader(unlabeled_set, batch_size=batch_size, shuffle=True, num_workers=2)
test_loader = torch.utils.data.DataLoader(testset, batch_size=batch_size, shuffle=False, num_workers=2)

Files already downloaded and verified
Files already downloaded and verified


In [21]:
# Define a simple CNN model
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16 * 5 * 5)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

# Instantiate the model, loss, and optimizer
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)
#model = Net().to(device)
model = resnet18(pretrained=False, num_classes=10).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

# Training loop
num_epochs = 50
lambda_ = 0.1

for epoch in range(num_epochs):
    print(epoch)
    running_loss = 0.0
    for i, ((labeled_data, labeled_targets), (unlabeled_data, _)) in enumerate(zip(labeled_loader, unlabeled_loader)):
        labeled_data, labeled_targets = labeled_data.to(device), labeled_targets.to(device)
        unlabeled_data = unlabeled_data.to(device)

        optimizer.zero_grad()

        # Forward pass for labeled data
        labeled_outputs = model(labeled_data)
        supervised_loss = criterion(labeled_outputs, labeled_targets)

        # Forward pass for unlabeled data
        unlabeled_outputs = model(unlabeled_data)
        probs = F.softmax(unlabeled_outputs, dim=1)
        entropy_loss = -torch.sum(probs * F.log_softmax(unlabeled_outputs, dim=1), dim=1).mean()

        # Combine supervised loss and entropy loss
        total_loss = supervised_loss + lambda_ * entropy_loss

        # Backward pass and optimization
        total_loss.backward()
        optimizer.step()

        # Print statistics
        running_loss += total_loss.item()
        if i % 2000 == 1999:
            print('[%d, %5d] loss: %.3f' % (epoch + 1, i + 1, running_loss / 2000))
            running_loss = 0.0

print('Finished Training')

cpu
0


c:\Users\mraji855\Desktop\Semi Supervised - DL\.venv\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\mraji855\Desktop\Semi Supervised - DL\.venv\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


KeyboardInterrupt: 

In [ ]:
# Evaluation
correct = 0
total = 0
with torch.no_grad():
    for data in test_loader:
        images, labels = data
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print('Accuracy of the network on the 10000 test images: %d %%' % (100 * correct / total))

Accuracy of the network on the 10000 test images: 54 %


In [25]:
import torch
import torchvision
import torchvision.transforms as transforms
from torchvision.models import resnet18
import torch.nn as nn
import torch.optim as optim

# Preprocessing: normalization and data augmentation
transform_train = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomCrop(32, padding=4),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
])

transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
])

# Load the CIFAR-10 dataset
trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform_train)
testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform_test)

# Split the dataset into labeled and unlabeled data
labeled_indices = list(range(0, 25000))
unlabeled_indices = list(range(25000, 50000))

labeled_set = torch.utils.data.Subset(trainset, labeled_indices)
unlabeled_set = torch.utils.data.Subset(trainset, unlabeled_indices)

# Create data loaders
batch_size = 100
labeled_loader = torch.utils.data.DataLoader(labeled_set, batch_size=batch_size, shuffle=True, num_workers=2)
unlabeled_loader = torch.utils.data.DataLoader(unlabeled_set, batch_size=batch_size, shuffle=True, num_workers=2)
test_loader = torch.utils.data.DataLoader(testset, batch_size=batch_size, shuffle=False, num_workers=2)

# Instantiate the model, loss, and optimizer
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = resnet18(pretrained=False, num_classes=10).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

# Training loop
num_epochs = 50
lambda_ = 0.1

for epoch in range(num_epochs):
    print(epoch)
    running_loss = 0.0
    for i, ((labeled_data, labeled_targets), (unlabeled_data, _)) in enumerate(zip(labeled_loader, unlabeled_loader)):
        labeled_data, labeled_targets = labeled_data.to(device), labeled_targets.to(device)
        unlabeled_data = unlabeled_data.to(device)

        optimizer.zero_grad()

        # Forward pass for labeled data
        labeled_outputs = model(labeled_data)
        supervised_loss = criterion(labeled_outputs, labeled_targets)

        # Forward pass for unlabeled data
        unlabeled_outputs = model(unlabeled_data)
        probs = F.softmax(unlabeled_outputs, dim=1)
        entropy_loss = -torch.sum(probs * F.log_softmax(unlabeled_outputs, dim=1), dim=1).mean()

        # Combine supervised loss and entropy loss
        total_loss = supervised_loss + lambda_ * entropy_loss

        # Backward pass and optimization
        total_loss.backward()
        optimizer.step()

        # Print statistics
        running_loss += total_loss.item()
        if i % 2000 == 1999:
            print('[%d, %5d] loss: %.3f' % (epoch + 1, i + 1, running_loss / 2000))
            running_loss = 0.0

print('Finished Training')

# Evaluation
correct = 0
total = 0
with torch.no_grad():
    for data in test_loader:
        images, labels = data
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print('Accuracy of the network on the 10000 test images: %d %%' % (100 * correct / total))



Files already downloaded and verified
Files already downloaded and verified
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
Finished Training
Accuracy of the network on the 10000 test images: 70 %
